In [2]:
import tweepy
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '' if t.startswith('#') and len(t) > 1 else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [ ]:
ids_df = pd.read_csv('../Data/corpus_marcado6emociones.txt', sep=' ',skiprows=19, header=None, names = ['id','Emocion']).apply(lambda s:s.str.replace("'", ""))

In [ ]:
inicio = 0
ids_list = []
emocion_list = pd.Series(dtype=str)
text_list = []
for i in list(range(100,len(ids_df),100)) + [len(ids_df)]:
    ids_list_temp = list(ids_df['id'][inicio:i])
    data = client.get_tweets(ids = ids_list_temp).data
    if data != None:
        text_list = text_list + [j.text for j in data ]
        ids_list_filtr = [str(i.id) for i in data ]
        emocion_list = emocion_list.append(ids_df[inicio:i][ids_df['id'][inicio:i].isin(ids_list_filtr)]['Emocion'].reset_index(drop = True))
        ids_list = ids_list + ids_list_filtr
    inicio = i
    print(i)
emociones_tweets_df = pd.DataFrame({'id':ids_list, 'text':text_list, 'emocion': emocion_list.reset_index(drop = True) })

In [ ]:
emociones_tweets_df.to_csv('../Data/Sidorov/corpus_marcado6emociones_extracted.csv', index=False)  

In [3]:
emociones_tweets_df = pd.read_csv('../Data/Sidorov/corpus_marcado6emociones_extracted.csv')

In [29]:
emociones_tweets_df = emociones_tweets_df[emociones_tweets_df['emocion'].notnull()].reset_index()

In [5]:
map_dict = {0:"ira", 1:"miedo", 2:"tristeza", 3:"alegria", 4:"asco", 5:"sorpresa", 6:"otra"}
map_dict_inverse = {v: k for k, v in map_dict.items()}

In [34]:
emociones_tweets_df['label'] =emociones_tweets_df["emocion"].str.lower().map(map_dict_inverse)

In [36]:
emociones_tweets_df.to_csv('../Data/Sidorov/corpus_marcado6emociones_extracted.csv', index=False)  

In [25]:
emociones_tweets_df['text'] = emociones_tweets_df['text'].apply(preprocess)
emociones_tweets_df.to_csv('../Data/Sidorov/sidorov_clean.csv', index=False)  

In [26]:
train, test = train_test_split(emociones_tweets_df[['text', 'label']], test_size=0.2, random_state=42)

In [33]:
train.to_csv('../Data/Huggingface/sidorov_hugging_train.csv', index=False) 
test.to_csv('../Data/Huggingface/sidorov_hugging_test.csv', index=False)  

In [17]:
emociones_tweets_df['text'][2000]

'Todos dios intenta dar una buena imagen por las redes sociales.... y luego si los vieras en persona te llevas disgustos #asco'

In [32]:
train

,text,label
1699,"Ariel Freez Lo más importante en la vida, es a...",3
1379,"Tener una amiga de fierro, un amor sincero y u...",3
180,"Pachuca a la final, Delfines campeon, mañana c...",3
5632,una mujer poca ropa riendo como loca para lla...,0
6874,Récord Mundial: Mayor rompecabezas hecho con t...,5
...,...,...
5734,"""VIENE LA VELA,ME DIJERON LOS DE ARRIBA QUE L...",0
5191,"las personas que critican todo el tiempo, a t...",0
5390,la gente que habla mal a espaldas de otros mi...,0
860,Que bueno saber que Tus Palabras pueden marcar...,3
